## System 5: Prediction and Modeling

This system takes the combined operations and adds useful metadata to each operation to account for things like APU time vs 400Hz time for a given operation, labeling of operation success, and also emissions.

**Inputs:**

- 4a: Combined data with each operation listed as metadata on each power consumption time block
- 5a: Mapping from Aircraft Model to corresponding APU group and ADG categories
- 5b: Mapping from APU group to corresponding emissions factors

**Output:**

- 5c: Final data CSV with operations and all emissions factors.

In [1]:
# Dependencies

# For any missing libraries, just run (remove curly braces):
# !pip install {library_name}
 
## Required
import pandas as pd
import datetime as dt
import numpy as np
from tqdm import tqdm_notebook as tqdm
import sklearn.model_selection
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [2]:
# Inputs, settings, and toggles

# Consumption Data Granularity to Filter On (operations with mixed granularity or other granularities will be discarded)
# In minutes, default value = 5
desired_granularity_mins = 5

# For the granularity above, define a minimum consumption value (kW) that indicates whether 400Hz power was used
# over that time period or not (> this threshold implies at least some 400Hz was drawn)
# in kW, default value = 2
threshold_kW = 2

# Location of input data from System 4
input_4a_location = 'sample_data/4a_combined_operations_and_consumption_data.csv'

# Location of mapping from Aircraft Model to corresponding APU group and ADG categories
input_5a_location = 'sample_data/5a_model_to_emissions_group.csv'

# Location of mapping from APU group to corresponding emissions factors
input_5b_location = 'sample_data/5b_emissions_group_to_emission_vals.csv'

# Desired location of output data
output_5c_location = 'sample_data/5c_all_operations_with_all_emissions_metadata.csv'

In [3]:
# Bring in combined operations & consumption data from System 4
date_like = ['Real_Timestamp', 'Actual Landing Time (Aerobahn)', 
             'Scheduled In Block Time (Aerobahn)',
             'Scheduled Off Block Time (Aerobahn)',
             'in_block_time_dt',
             'off_block_time_dt', 
             'Row_Time_Delta']
data = pd.read_csv(input_4a_location, parse_dates=date_like,  infer_datetime_format=True)
print("Imported CSV with %d rows" % len(data))
data = data.sort_values(by=['Gate', 'Real_Timestamp'])

# Row time delta handling, conversion to NS
data['Row_Time_Delta'] = pd.to_timedelta(data['Row_Time_Delta']) # Force Time Delta column type
data['Row_Time_Delta_ns'] = data['Row_Time_Delta'].values.astype(np.int64) # Convert to NS

Imported CSV with 91506 rows


In [4]:
# Group rows that represent a single operation together, and define an op_id column
group_vars = ['Gate', 'Actual Landing Time (Aerobahn)', 'Registration'] # Choose columns to group by to define an operation
data['op_id'] = data.groupby(group_vars).ngroup() # Define a unique op id (unique in the space of this dataset)
data.reset_index(drop=True, inplace=True)

In [5]:
# Filter out operations that don't contain only rows with the desired granularity

data['granularity_valid'] = False
print("Starting with %d operations" % len(data))
print("Filtering out all operations that don't have desired granularity: %d mins" % desired_granularity_mins)
def apply_5m_column(grp):
    val = np.mean(grp['Row_Time_Delta_ns'])
    # 1 minute = 60000000000 ns
    if val == (60000000000 * desired_granularity_mins): # 300000000000 = 5 min
        grp['only_5m'] = True
    return grp
filtered_data = data.groupby('op_id').apply(apply_5m_column)
data = filtered_data[filtered_data['only_5m'] == True]
print("Ended up with with %d operations that meet granularity criteria" % len(data))

Starting with 91506 operations
Filtering out all operations that don't have desired granularity: 5 mins


/Users/chanan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/groupby/groupby.py:937: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = concat(values, axis=self.axis)


Ended up with with 88277 operations that meet granularity criteria


In [7]:
# Re-do op_id calculation with fewer number of operations

data['op_id'] = data.groupby(group_vars).ngroup()

# Add column with threshold minimum as defined above 
# (minimum kW consumption value for a row to be considered to be drawing any 400Hz power)
data['threshold'] = threshold_kW

# Add greater than threshold column for rows with power over our threshold value
data['>threshold'] = data['Power_kW'] > data['threshold']

/Users/chanan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/chanan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/chanan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [10]:
data_for_prediction = data[data['>threshold'] == 1]
# Note: One may consider adding numerical variables as features. We found that using 
# airplane model was sufficient, but we left in the ability to add additional 
# categorical and numberical variables to the model. Just uncomment the commented out lines

# num_vars = ['example_numerical_col1', 'example_numerical_col2']
cat_vars = ['Model'] 

    
train, test = sklearn.model_selection.train_test_split(data_for_prediction, 
                                                       train_size=0.8, 
                                                       test_size=0.2, 
                                                       random_state=5)

print('Train:', train.shape, 'Test:', test.shape)

#scaler = StandardScaler()
#scaler.fit(train[num_vars])

# Sourced from UC Berkeley, Data 100, Fall 2019
def design_matrix(t):
    #scaled = t[num_vars].copy()
    #scaled.iloc[:,:] = scaler.transform(scaled) # Convert to standard units
    categoricals = [pd.get_dummies(t[s], prefix=s, drop_first=True) for s in cat_vars]
    return pd.concat(categoricals, axis=1)

def rmse(errors):
    """Return the root mean squared error."""
    return np.sqrt(np.mean(errors ** 2))    

model = LinearRegression()
train2 = design_matrix(train)
test2 = design_matrix(test)
og2 = design_matrix(data)

def add_missing_cols(og, dataset):
    # Get missing columns in the training test
    missing_cols = set(og.columns) - set(dataset.columns)
    # Add a missing column in test set with default value equal to 0
    for c in missing_cols:
        dataset[c] = 0
    # Ensure the order of column in the test set is in the same order than in train set
    dataset, og = dataset.align(og, axis=1)
    return dataset

train3 = add_missing_cols(og2, train2)
test3 = add_missing_cols(og2, test2)

#train3, test3 = train2.align(test2, join='outer', axis=1, fill_value=0)

model.fit(train3, train['Power_kW'])
model_prediction = model.predict(test3)
linear_rmse = rmse(test['Power_kW'] - model_prediction)
test['Power_Predicted'] = model_prediction
print("Our mean squared error is %f " % linear_rmse)

model.fit(add_missing_cols(og2, design_matrix(data_for_prediction)), data_for_prediction['Power_kW'])
data['prediction'] = model.predict(design_matrix(data))

Train: (31705, 29) Test: (7927, 29)
Our mean squared error is 8.945778 


/Users/chanan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/chanan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
# Build a ratio of power over prediction
data['power_over_pred'] = data['Power_kW'] / data['prediction']

# Identify where the 400hz ground power is being used and where it is not. 
# When power>threshold, the 400Hz ratio is 1. For edge cases, after a switch on just occured 
# and before a switch off, the 400Hz ratio is estimated by observing the ratio between the actual and the 
# predicted power. Example: at time 1:00, threshold>power, at time 1:05, theshold<power=8kw but the 
# predicted power was 20 kW. Therefore 8/20=40%=2 minutes are estimated to be connected. Therefore, we 
# estimate the switch occured at 1:03.
def compute_ratio_400hz(grp):
    # add switch
    last_val = False
    for index, row in grp.iterrows():
        if row['>threshold'] and not last_val: #switch
            grp.loc[index, 'switch'] = True
            grp.loc[index, 'ratio_400hz'] = np.minimum(1.0, grp.loc[index, 'power_over_pred'])
        elif not row['>threshold'] and last_val: # switch
            if index > 0:
                grp.loc[index - 1, 'switch'] = True
                grp.loc[index - 1, 'ratio_400hz'] = np.minimum(1.0, grp.loc[index - 1, 'power_over_pred'])
            grp.loc[index, 'switch'] = False
            grp.loc[index, 'ratio_400hz'] = int(grp.loc[index, '>threshold']) # case where switch is False
        else:
            grp.loc[index, 'switch'] = False
            grp.loc[index, 'ratio_400hz'] = int(grp.loc[index, '>threshold']) # case where switch is False
        last_val = row['>threshold']
        
    return grp

data_with_ratios = data.groupby('op_id').apply(compute_ratio_400hz)

/Users/chanan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
# Now that we know when the switches occur, define ratios for 400Hz and APU

data_with_ratios['ratio_APU'] = 1.0 - data_with_ratios['ratio_400hz'] 
data_with_ratios['time_400hz'] = data_with_ratios['ratio_400hz'] * 5 * data_with_ratios['plane_in_ratio']
data_with_ratios['time_APU'] = data_with_ratios['ratio_APU'] * 5 * data_with_ratios['plane_in_ratio']
data_with_ratios['num_switches'] = 0
data_with_ratios['total_400hz_time'] = 0
data_with_ratios['total_APU_time'] = 0

def calc_some_totals(op):
    num_switches = np.sum(op['switch'])
    time_400hz = np.sum(op['time_400hz'])
    time_APU = np.sum(op['time_APU'])
    op['num_switches'] = num_switches
    op['total_time_400hz'] = time_400hz
    op['total_time_APU'] = time_APU
    return op

data_with_ratios.reset_index(drop=True, inplace=True)
data_with_ratios_and_sums = data_with_ratios.groupby('op_id').apply(calc_some_totals)

In [13]:
data = data_with_ratios_and_sums
data['ratio_400hz'] = data['ratio_400hz'] * data['plane_in_ratio']
data['ratio_APU'] = data['ratio_APU'] * data['plane_in_ratio']
data = data.dropna(subset=['Gate', 'Carrier Group', 'Model'])

In [14]:
def compute_emissions(grp):
    
    total_real = 0
    total_400hz = 0
    total_APU = 0
    
    for item, row in grp.iterrows():
        chunk_time_in_hours = (int(row['Row_Time_Delta_ns']) / 60000000000) / 60
        total_real += row['Power_kW'] * chunk_time_in_hours
        total_400hz += row['prediction'] * row['ratio_400hz'] * chunk_time_in_hours
        total_APU += row['prediction'] * row['ratio_APU'] * chunk_time_in_hours
    
    total_pred = total_400hz + total_APU
    total_pred_ratio = 0
    if (total_real != 0):
        total_pred_ratio = (total_real - total_400hz) / total_real        
    
    grp['op_total_real'] = total_real
    grp['op_total_400hz'] = total_400hz
    grp['op_total_APU'] = total_APU
    grp['op_total_pred'] = total_pred
    grp['op_total_pred_ratio'] = total_pred_ratio

    return grp

data_with_totals = data.groupby('op_id').apply(compute_emissions)

In [15]:
# Add APU type and AAC and ADG to each row
apu_group_by_model = pd.read_csv(input_5a_location, low_memory=False)
apu_group_by_model = apu_group_by_model.set_index('Subtype')
data_with_groups = data_with_totals.join(other=apu_group_by_model, on='Model', how='left')

# Add emissions data by APU type 
emissions_by_apu_group = pd.read_csv(input_5b_location, low_memory=False)
emissions_by_apu_group = emissions_by_apu_group.set_index('APU Group')
data_with_emissions = data_with_groups.join(other=emissions_by_apu_group, on='Emissions_Group', how='left')

# Define number of APU starts using switch data
data_with_emissions['num_starts'] = 1 + (data_with_emissions['num_switches'] // 2)
data_with_emissions['apu_start_time'] = data_with_emissions['num_starts'] * 3

# Bring in columns from emissions data and convert from hours to minutes 
cols = ['Normal NOx (kg/hr)', 
        'Normal NO2 (kg/hr)', 
        'Normal CO (kg/hr)', 
        'Normal HC (kg/hr)', 
        'Normal PM (kg/hr)', 
        'Normal Fuel (kg/hr)']
for col in cols:
    col_new_name = col[:-7]
    col_new_name = col_new_name + "(kg)"
    data_with_emissions[col_new_name] = (data_with_emissions[col] * data_with_emissions['time_APU']) / 60

# Aggregate emissions data on a per-operation level
cols_to_agg = ['op_id']
for col in cols:
    col_new_name = col[:-7] + "(kg)"
    cols_to_agg.append(col_new_name)
    
emissions_per_chunk = data_with_emissions[cols_to_agg]
emissions_per_chunk = emissions_per_chunk.set_index('op_id')

emissions_agg = emissions_per_chunk.groupby('op_id').agg('sum')
emissions_joined = data_with_emissions.join(emissions_agg, on='op_id', lsuffix="_row", rsuffix="_total")

# Do the same process as above, except with the APU start emissions factors
cols2 = ['Start NOx (kg/hr)', 'Start NO2 (kg/hr)', 'Start CO (kg/hr)', 'Start HC (kg/hr)', 'Start PM (kg/hr)', 'Start Fuel (kg/hr)']

for col in cols2: 
    col_without_hr = col[:-7] + "(kg)_total"
    emissions_joined[col_without_hr] = (emissions_joined[col] * emissions_joined['apu_start_time']) / 60
    
cats = ['NOx', 'NO2', 'CO', 'HC', 'PM', 'Fuel']
for cat in cats:
    normal_total_col = "Normal " + cat + " (kg)_total"
    start_total_col = "Start " + cat + " (kg)_total"
    result_col = cat + " (kg)_grandtotal"
    emissions_joined[result_col] = emissions_joined[normal_total_col] + emissions_joined[start_total_col]
    
# Add a successful metric to each operation    
emissions_joined.reset_index(drop=True, inplace=True)    
emissions_joined['row_successful'] = emissions_joined['Power_kW'] >= (emissions_joined['plane_in_ratio'] * emissions_joined['threshold'])
def apply_ratio_successful(grp):
    num_success = len(grp[grp['row_successful'] == True])
    num_tot = len(grp)
    grp['ratio_successful'] = num_success / num_tot
    return grp
emissions_joined = emissions_joined.groupby('op_id').apply(apply_ratio_successful)
    
print("Exporting to CSV...")    
export_csv = emissions_joined.to_csv(output_5c_location, index = None, header=True)
print("Exporting complete!")    

Exporting to CSV...
Exporting complete!
